# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *


In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
# Get a list of all JSON files in data/song_data
song_files = get_files("data/song_data")

In [5]:
# Select the first song from this list
filepath = song_files[0]
filepath

'/home/workspace/Project-1 Data Modelling with Postgres/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [6]:
# Read the song file and view the data
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
# Select the required data values for songs table from the df 
song_data = list(df[["song_id", "title", "artist_id", "year", "duration"]].values[0])
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
# Insert the data from song_data list to the songs table
cur.execute(song_table_insert, song_data)
conn.commit()

In [9]:
# View the data in the songs table
cur.execute("SELECT * FROM songs LIMIT 5")
row = cur.fetchone()
while(row):
    print("row = ", row)
    row = cur.fetchone()

row =  ('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, Decimal('218.93179'))
row =  ('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, Decimal('266.39628'))
row =  ('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, Decimal('269.81832'))
row =  ('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, Decimal('189.57016'))
row =  ('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, Decimal('148.03546'))


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
# Select the required data values for artists table from the df 
artist_data = list(df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
# Insert the data from artist_data list to the artists table
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [12]:
# View the data in the artists table
cur.execute("SELECT * FROM artists LIMIT 5")
row = cur.fetchone()
while(row):
    print("row = ", row)
    row = cur.fetchone()

row =  ('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', Decimal('NaN'), Decimal('NaN'))
row =  ('ARNTLGG11E2835DDB9', 'Clp', '', Decimal('NaN'), Decimal('NaN'))
row =  ('AR8ZCNI1187B9A069B', 'Planet P Project', '', Decimal('NaN'), Decimal('NaN'))
row =  ('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', Decimal('NaN'), Decimal('NaN'))
row =  ('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', Decimal('35.14968'), Decimal('-90.04892'))


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
# Get a list of all JSON files in data/log_data
log_files = get_files("data/log_data")

In [14]:
# Select the first song from this list
filepath = log_files[0]
filepath

'/home/workspace/Project-1 Data Modelling with Postgres/data/log_data/2018/11/2018-11-30-events.json'

In [15]:
# Read the log file and view the data
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
# Filter records by NextSong action
df = df[df["page"]=="NextSong"]

In [17]:
# Convert the ts column to datetime
t = pd.to_datetime(df['ts'])
t.head()

0   1970-01-01 00:25:43.537327796
1   1970-01-01 00:25:43.540121796
2   1970-01-01 00:25:43.540368796
3   1970-01-01 00:25:43.540625796
4   1970-01-01 00:25:43.540856796
Name: ts, dtype: datetime64[ns]

In [18]:
# Extract the required values from the ts column in df and create a time_data list
time_data = list()

for row in t:
    time_data.append([row, row.hour, row.day, row.week, \
                      row.month, row.year, row.dayofweek])

In [19]:
# Create a list of column_names
column_labels = ["timestamp", "hour", "day", "week_of_year", "month", "year", "weekday"]

In [20]:
# Create a time_df from time_data list and column_labels
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1970-01-01 00:25:43.537327796,0,1,1,1,1970,3
1,1970-01-01 00:25:43.540121796,0,1,1,1,1970,3
2,1970-01-01 00:25:43.540368796,0,1,1,1,1970,3
3,1970-01-01 00:25:43.540625796,0,1,1,1,1970,3
4,1970-01-01 00:25:43.540856796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
# Insert the data in the time table
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [22]:
# View the data in the time table
cur.execute("SELECT * FROM time LIMIT 5")
row = cur.fetchone()
while(row):
    print("row = ", row)
    row = cur.fetchone()

row =  (datetime.datetime(1970, 1, 1, 0, 25, 43, 537328), 0, 1, 1, 1, 1970, 3)
row =  (datetime.datetime(1970, 1, 1, 0, 25, 43, 540122), 0, 1, 1, 1, 1970, 3)
row =  (datetime.datetime(1970, 1, 1, 0, 25, 43, 540369), 0, 1, 1, 1, 1970, 3)
row =  (datetime.datetime(1970, 1, 1, 0, 25, 43, 540626), 0, 1, 1, 1, 1970, 3)
row =  (datetime.datetime(1970, 1, 1, 0, 25, 43, 540857), 0, 1, 1, 1, 1970, 3)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [23]:
# Select the required data from the df for the users table
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [24]:
# Insert the data in the users table
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [25]:
# View the data in the users table
cur.execute("SELECT * FROM users LIMIT 5")
row = cur.fetchone()
while(row):
    print("row = ", row)
    row = cur.fetchone()

row =  (86, 'Aiden', 'Hess', 'M', 'free')
row =  (57, 'Katherine', 'Gay', 'F', 'free')
row =  (30, 'Avery', 'Watkins', 'F', 'paid')
row =  (74, 'Braden', 'Parker', 'M', 'free')
row =  (92, 'Ryann', 'Smith', 'F', 'free')


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
for index, row in df.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results

    else:
        songid, artistid = None, None
    
     # insert songplay record
    songplay_data = (pd.to_datetime(row.ts),\
                     row.userId, row.level, songid, artistid, row.sessionId, \
                     row.location, row.userAgent)


    cur.execute(songplay_table_insert, songplay_data)

In [27]:
# View the data in the songplays table
cur.execute("SELECT * FROM songplays LIMIT 5")
row = cur.fetchone()
while(row):
    print("row = ", row)
    row = cur.fetchone()

row =  (1, datetime.datetime(1970, 1, 1, 0, 25, 43, 537328), 91, 'free', None, None, 829, 'Dallas-Fort Worth-Arlington, TX', 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)')
row =  (2, datetime.datetime(1970, 1, 1, 0, 25, 43, 540122), 73, 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
row =  (3, datetime.datetime(1970, 1, 1, 0, 25, 43, 540369), 73, 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
row =  (4, datetime.datetime(1970, 1, 1, 0, 25, 43, 540626), 73, 'paid', None, None, 1049, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
row =  (5, datetime.datetime(1970, 1, 1, 

Run `test.ipynb` to see if you've successfully added records to this table.

In [28]:
# View the data in the songplays table having non-null values for song_id and artist_id
cur.execute("SELECT * FROM songplays WHERE song_id is not null and artist_id is not null LIMIT 5")
row = cur.fetchone()
while(row):
    print("row = ", row)
    row = cur.fetchone()

row =  (4108, datetime.datetime(1970, 1, 1, 0, 25, 42, 837408), 15, 'paid', 'SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4', 818, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')


# Close Connection to Sparkify Database

In [29]:
# Closing the database connection
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.